# Update profiles for functional groups of the IUCN Global Ecosystem typology

Scripts by José R. Ferrer-Paris

The scripts described in this document are used to:

- Read data from database
- Write static markdown pages for a Jekyll site

## Set-up
Load all the libraries we will need in this script:

In [60]:
import os
from pathlib import Path
import re
from datetime import datetime
from configparser import ConfigParser
import psycopg2
from psycopg2.extras import DictCursor
from psycopg2.extensions import AsIs
import shutil

Read configuration parameters for the connection to the current version of the database:

In [2]:
filename = Path(os.path.expanduser('~')) / ".database.ini"
section = 'psqlaws'

parser = ConfigParser()
parser.read(filename)
db = {}
if parser.has_section(section):
    params = parser.items(section)
    for param in params:
        db[param[0]] = param[1]
else:
    raise Exception('Section {0} not found in the {1} file'.format(section, filename))

params = db

## Copy DAM between repos
Assumming  `repo one` has been updated recently, copy files to `repo two`:

In [27]:
repodir = Path(os.path.expanduser('~')) / 'proyectos' / 'typology-website'
repo_one = repodir  / 'typology-map-content' / 'assets' / 'uploads'
repo_two =  repodir / 'Ecosystem-profiles-comments' / 'assets' / 'diagrams'

DAMfiles=list()
for fn in os.listdir(repo_one):
    if fn.find('diagram')>0:
        DAMfiles.append(fn)

In [43]:
for infile in DAMfiles:
    nfn = infile.replace('-diagram.png','')
    outfile = re.sub(r'([mfts]+)_([0-9])_([0-9])',r'\1\2.\3',nfn).upper() +'.png'
    source =  repo_one / infile
    destination =  repo_two / outfile
    # copy only files
    if os.path.isfile(source):
        shutil.copy(source, destination)
    

## Update content to version 2.1 

In [85]:
conn = psycopg2.connect(**params)
cur = conn.cursor(cursor_factory=DictCursor)

In [86]:
qry="SELECT distinct code FROM functional_groups LIMIT 2;"
cur.execute(qry)
efgs = cur.fetchall()


In [98]:
efgs

[['F1.1'], ['F1.2']]

In [111]:
sections = ("Ecological Traits","Key Ecological Drivers","Distribution")
version = 'v2.1'

for efg in efgs:
    qry="""
    SELECT code,f.biome_code,f.name,b.name as biome_name,realms,f.update as original_date 
    FROM functional_groups f
    LEFT JOIN biomes b
        ON b.biome_code=f.biome_code
    WHERE code = %s
    ;"""
    cur.execute(qry,(efg[0],))

    efginfo = cur.fetchone()
    record=dict()
    
    record['name']=efginfo['name']
    record['biome']=efginfo['biome_name']
    record['realm']=efginfo['realms']
    record['code']=efginfo['code']
    record['biomecode']=efginfo['biome_code']
    record['contributors']=list()
    record['mapcontributors']=list()
    
    for section in sections:
        qry ="""
        SELECT description,contributors,version,update 
        FROM efg_%s 
        WHERE code = %s AND language = 'en' AND version=%s
        ORDER BY update DESC;
        """
        cur.execute(qry,(AsIs(section.lower().replace(' ','_')),efginfo['code'],version))
        sectinfo = cur.fetchone()
        record[section]=sectinfo['description']
        for author in sectinfo['contributors']:
            if author not in record['contributors']:
                record['contributors'].append(author)
        record['version']="%s (%s)" % (sectinfo['version'],sectinfo['update'].date())
    
    qry="""
    SELECT map_code,map_source,contributors,map_version 
    FROM map_metadata WHERE code = %s AND status='valid' AND map_type='Indicative Map' 
    ORDER BY map_version DESC;
    """
    cur.execute(qry,(efg[0],))
    mapinfo = cur.fetchone()
    for author in mapinfo['contributors']:
        if author not in record['mapcontributors']:
            record['mapcontributors'].append(author)
    record['Map description']=mapinfo['map_source']
    record['mapversion']="%s_%s" % (mapinfo['map_code'],mapinfo['map_version'])

In [113]:
record

{'name': 'F1.2 Permanent lowland rivers',
 'biome': 'F1. Rivers and streams biome',
 'realm': ['Freshwater'],
 'code': 'F1.2',
 'biomecode': 'F1',
 'contributors': ['RT Kingsford',
  'R Mac Nally',
  'PS Giller',
  'MC',
  'Rains',
  'AH Arthington',
  'DA Keith'],
 'mapcontributors': ['JR Ferrer-Paris', 'DA Keith'],
 'Ecological Traits': 'Small-medium lowland rivers (stream orders 4-9) are productive depositional ecosystems with trophic webs that are less diverse than large lowland rivers (F1.7). Macrophytes rooted in benthos or along the river margins contribute most primary production, but allochthonous inputs from floodplains and upper catchments generally dominate energy flow in the system. The biota tolerates a range of temperatures, which vary with catchment climate. Aquatic biota have physiological, morphological and even behavioural adaptations to lower oxygen concentrations, which may vary seasonally and diurnally. Zooplankton can be abundant in slower deeper rivers. Sessile 

In [94]:
template = """
---
name: {name}
biome: {biome}
realm: {realms}
code: {code}
biomecode: {biomecode}
contributors: {}
mapcontributors: {}
version: {}
---
# Ecosystem properties

{ecological_traits}

# Ecological Drivers

{key_ecological_drivers}

# Diagramatic assembly model
 
{% include DAM.html %}
 
# Distribution

{distribution}

"""

In [97]:
sectinfo

['Distributed throughout tropical and temperate lowlands but very uncommon in arid zones. They are absent from boreal zones, where they are replaced by F1.3.',
 ['RT Kingsford',
  'R Mac Nally',
  'PS Giller',
  'MC',
  'Rains',
  'AH Arthington',
  'DA Keith'],
 'v2.1',
 datetime.datetime(2022, 4, 6, 5, 2, 1)]

In [84]:

cur.close()
        
if conn is not None:
    conn.close()
    print('Database connection closed.')

Database connection closed.


In [ ]:

   ## Query text section

  

   efg.texts <- rbind(efg.texts,data.frame(section="Diagramatic assembly model",description="{% include DAM.html %}",version=NA,update=NA))

   ## Query map info
   qry <- sprintf(, target.EFG)
      efg.maps <-   dbGetQuery(con,qry)

   map.authors <- strsplit(gsub("\\{|\\}|\"","",efg.maps$contributors),",")[[1]]


   ## Query References

   qry <- sprintf("SELECT ref_cite  FROM efg_references as e LEFT JOIN ref_list as l ON e.ref_code=l.ref_code WHERE code = '%s' ", target.EFG, max(efg.texts$version,na.rm=T))
      text.references <-   dbGetQuery(con,qry)$ref_cite


      qry <- sprintf("SELECT ref_cite  FROM map_references as e LEFT JOIN ref_list as l ON e.ref_code=l.ref_code WHERE map_code = '%s' AND map_version = '%s'", efg.maps$map_code, max(efg.maps$map_version,na.rm=T))
         map.references <-   dbGetQuery(con,qry)$ref_cite





## output header
cat(file=target.arch,sprintf("---
name: %1$s
biome: %8$s
realm: %9$s
code: %2$s
biomecode: %3$s
contributors: %4$s
mapcontributors: %7$s
version: %5$s, %6$s
---",
efg.info$name,
efg.info$code,
efg.info$biome_code,
paste(authors,collapse=", "),
max(efg.texts$version,na.rm=T),
max(efg.texts$update,na.rm=T),
paste(map.authors,collapse=", "),
efg.info$biome_name,
paste(strsplit(gsub("\\{|\\}|\"","",efg.info$realms),",")[[1]],collapse=", ")

))

## output sections
cat(file=target.arch,sprintf("\n# %s\n \n%s\n", efg.texts$section[c(1,2,4,3)], efg.texts$description[c(1,2,4,3)]), append=T)

## output map info

cat(file=target.arch, sprintf("
{%% capture map_det %%}
%s
{%% endcapture %%}
{%% include MAP.html %%}
",efg.maps$map_source), append=T)

## output references

cat(file=target.arch,sprintf("
## References
### Main References
%s
### Map References
%s
",paste("*",text.references,sep=" ",collapse="\n"),paste("*",map.references,sep=" ",collapse="\n")), append=T)

}

dbDisconnect(con)


In [22]:
## Add maps with version codes

'M_3_7.PNG'

In [23]:
## Add map descriptions

'm_3_7-diagram.png'